<a href="https://colab.research.google.com/github/AhmadIsmail83N/MyData/blob/master/UC_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


**Import Libraries**

In [52]:
from packaging import version

import tensorflow as tf
from tensorflow import keras
import datetime as dt
                                      
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.2.0-rc4


**download MNIST and create data set for training**

In [0]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices((x_train)).batch(32).shuffle(10000)
train_dataset = train_dataset.map(lambda x: (tf.math.divide(tf.cast(x, tf.float32), 255.0)))
#Бинаризация
train_dataset = train_dataset.map(lambda x: (tf.round(x)))
train_dataset = train_dataset.map(lambda x: (tf.image.random_flip_left_right(x)))

test_dataset = tf.data.Dataset.from_tensor_slices((x_test)).batch(32)
test_dataset = test_dataset.map(lambda x: (tf.math.divide(tf.cast(x, tf.float32), 255.0)))
#Бинаризация
test_dataset = test_dataset.map(lambda x: (tf.round(x)))

**creating a variation auto-encoder.**

**Create Class Incoder: inheritance from Keras.Model**

method  _init : for creating model layers

method  call  : apply to Input x

In [0]:
class MnistEncoder(keras.Model):
    def __init__(self):
        super(MnistEncoder, self).__init__(name='mnist_encoder')
        with tf.device('/device:GPU:0'):
            
            
            self.Input =    keras.layers.InputLayer(input_shape=(28, 28))
            
            self.conv1 =    tf.keras.layers.Conv2D( 
                                                  filters=32, 
                                                   kernel_size=3,
                                                   strides=(2, 2),
                                                   activation='relu'
                                                  )
            self.conv2 =    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu')
            self.pool  =    tf.keras.layers.MaxPooling2D(strides=(2, 2), padding='same')
           
            self.flatten = keras.layers.Flatten(input_shape=(28, 28))
            self.fc2 = keras.layers.Dense(64,
                                         activation=tf.nn.relu,
                                         kernel_initializer=tf.initializers.RandomNormal,
                                         kernel_regularizer=keras.regularizers.l2(l=0.001))
            self.dropout = keras.layers.Dropout(0.5)
            self.fc_mu = keras.layers.Dense(2,
                                            activation=None,
                                            kernel_initializer=tf.initializers.RandomNormal,
                                            kernel_regularizer=keras.regularizers.l2(l=0.001))
            self.fc_var = keras.layers.Dense(2,
                                             activation=None,
                                             kernel_initializer=tf.initializers.RandomNormal,
                                             kernel_regularizer=keras.regularizers.l2(l=0.001))


    def call(self, x, training=False):
        if training:
            x = self.dropout(self.fc2(self.flatten(x)))
        else:
            x= self.flatten(x)
            x = self.conv1(x)
            x=  self.pool(x)
            x = self.conv2(x)
            x=  self.pool(x)
            x = self.fc2(x)
        mu = self.fc_mu(x)
        var = self.fc_var(x)
        return mu, var

**Decoder**

In [0]:
class MnistDecoder(keras.Model):
    def __init__(self):
        super(MnistDecoder, self).__init__(name='mnist_decoder')
        with tf.device('/device:GPU:0'):
            self.fc1 = keras.layers.Dense(64,
                                             activation=tf.nn.relu,
                                             kernel_initializer=tf.initializers.RandomNormal)        
    #         self.fc2 = keras.layers.Dense(256,
    #                                       activation=tf.nn.relu,
    #                                          kernel_initializer=tf.initializers.RandomNormal)
            self.Input= keras.layers.InputLayer(input_shape=(2,))

            self.dropout = keras.layers.Dropout(0.5)    
            self.Conv2DT1 = keras.layers.Conv2DTranspose(
                                                  filters=32,
                                                  kernel_size=3,
                                                  strides=(2, 2),
                                                  padding="SAME",
                                                  activation='relu')
            self.Conv2DT2 = keras.layers.Conv2DTranspose(
                                                  filters=64,
                                                  kernel_size=3,
                                                  strides=(2, 2),
                                                  padding="SAME",
                                                  activation='relu')            
            self.fc3 = keras.layers.Dense(784,
                                          activation=None,
                                             kernel_initializer=tf.initializers.RandomNormal)
            self.reshape = keras.layers.Reshape(target_shape=(28, 28))

    def call(self, x, training=False):
        x =self.fc1(x)
        if training:
            #x = self.dropout(self.fc2(x))
            x = self.dropout(self.fc3(x))
        else:
            #x = self.fc2(x)
            x=  self.input(x)           
            x = self.fc3(x)
            x=  self.reshape(x)
            x=  self.Conv2DT2(x)
            x=  self.Conv2DT1(x)        
        return x

**Create Model for use encoder and decoder**

In [0]:
class VAE(keras.Model):
    # Функции от keras.Model
        def __init__(self):
            super(VAE, self).__init__(name='mnist_vae')
            with tf.device('/device:GPU:0'):
                self.encoder = MnistEncoder()
                self.decoder = MnistDecoder()

        def call(self, x, training=False):
            mu, var = self.encoder(x,training)
            z = self.reparametrize(mu, var)
            x_hat = self.decoder(z,training)
            return x_hat, mu, var
            return x_hat, mu, var
    
    # Функция выполняющая репараметризацию
        def reparametrize(self, mu, var):
            eps = tf.random.normal(shape=mu.shape)
            return eps * var + mu
    
    # Функция выполняющая получение изображение из вектора z. Заметим, что 
    # здесь присутствует сигмоида
        def sample(self, eps=None):
            if eps is None:
                eps = tf.random.normal(shape=(100, self.latent_dim))
            return tf.sigmoid(self.decoder(eps,False))        

**Ready to train, build optimizer and loss function**

In [0]:
from datetime import datetime
import numpy as np

optimizer = tf.keras.optimizers.Adam(1e-4)

summary_writer = tf.summary.create_file_writer('logs', flush_millis=10000)
summary_writer.set_as_default()
@tf.function
def log_normal_pdf(mean, var, raxis=1):    
    arg1 = tf.multiply(tf.pow(mean,2) + tf.pow(var,2) - tf.math.log(tf.pow(var,2)) - 1,0.5)    
    return tf.reduce_sum(arg1,axis=1)

@tf.function
def compute_loss(model, x):
    x_hat, mu, var = model(x,False)
    ## сигмоида встроена в функцию потери
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_hat, labels=x)   
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2])   
    logqz_x = log_normal_pdf(mu, var)
    return -tf.reduce_mean(logpx_z  - logqz_x)

@tf.function
def compute_gradients(model, x):
    with tf.device('/gpu:0'):
        with tf.GradientTape() as tape:
            loss = compute_loss(model, x)
        return tape.gradient(loss, model.trainable_variables), loss
@tf.function
def apply_gradients(optimizer, gradients, variables):
    optimizer.apply_gradients(zip(gradients, variables))
    
def generate_and_save_images(model, epoch, test_input, file_writer):
    '''
    Генерирует примеры на основе test_input и записывает их в file_writer для
    визуализации в tensorboard
    '''
    predictions = model.sample(test_input)
    with file_writer.as_default():
        images = np.reshape(predictions, (-1, 28, 28, 1)) * 255
        images[images < 0] = 0
        images[images > 255] = 255
        tf.summary.image("16 generated data examples", images, max_outputs=16, step=epoch)

    for epoch in range(1, epochs + 1):
       start_time = time.time()
       for train_x in train_dataset:
          compute_gradients(model, train_x)
          with tf.GradientTape() as tape:
            loss = compute_loss(model, train_x)
            gradients = tape.gradient(loss, model.trainable_variables)
            apply_gradients(optimizer,gradients, model.trainable_variables)
          end_time = time.time()

          if epoch % 1 == 0:    
            loss = tf.keras.metrics.Mean()
            for test_x in test_dataset:
             loss(compute_loss(model, test_x))
          elbo = -loss.result()
          display.clear_output(wait=False)
          print('Epoch: {}, Test set ELBO: {}, '
              'time elapse for current epoch {}'.format(epoch,
                                                    elbo,
                                                    end_time - start_time))
          generate_and_save_images(
            model, epoch, random_vector_for_generation,file_writer) 
          
    

**Create instance of Model and start training**



# Загрузает расширение в Notebook
%load_ext tensorboard
....
# Открывает в выводе ячейки tensorboard
%tensorboard --logdir logs/gradient_tape

In [0]:
epochs = 100
latent_dim = 2
num_examples_to_generate = 16

# Создадим случайный вектор
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])

model = VAE()

In [0]:
import time

current_time  = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir  = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

generate_and_save_images(model, 0, random_vector_for_generation, test_summary_writer)
# Создаем метрики для вычисления.
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

%tensorboard --logdir logs/gradient_tape

for epoch in range(1, epochs + 1):
    for train_x in train_dataset:
        gradients, loss = compute_gradients(model, train_x)
        train_loss(loss)
        apply_gradients(optimizer, gradients, model.trainable_variables)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
    #print(epoch,loss)

    if epoch % 1 == 0:
#         for test_x in test_dataset:
#             test_loss(compute_loss(model, test_x))
#         with test_summary_writer.as_default():
#             tf.summary.scalar('loss', test_loss.result(), step=epoch)

        generate_and_save_images(model, epoch, random_vector_for_generation, test_summary_writer)
    # Обнуляем метрики на каждой эпохе
    train_loss.reset_states()
    test_loss.reset_states()

